在这次解密中 我将综合尝试使用字母频率分析和单词分析法 进行对单字母密码的解密

首先建立一个映射表，存储密钥的映射关系，以及对应的设置函数

In [33]:
dict = {}
def set_dict(keys, values):
    for i in range(len(keys)):
        dict[keys[i]] = values[i]

##### 词频分析 

![jupyter](image.png)

由图可知e出现次数最高,我们尝试对密文进行词频分析,找到出现次数最高的字母.

In [34]:


#大写转为小写
def convert_to_lower_case(string):
    return string.lower()

#是否为字母构成的字符串
def is_alpha(string):
    string = convert_to_lower_case(string)
    for i in range(len(string)):
        if string[i] not in 'abcdefghijklmnopqrstuvwxyz':
            return False
    return True

#打开ciphertext.txt文件
#分析每个字母的出现次数

counter = {}

with open('ciphertext.txt', 'r') as f:
    text = f.read()
    text = convert_to_lower_case(text)
    for i in range(len(text)):
        if is_alpha(text[i]) == 0:
            continue
        if text[i] in counter:
            counter[text[i]] += 1
        else:
            counter[text[i]] = 1
    counter = sorted(counter.items(), key=lambda x: x[1], reverse=True)
    print(counter)


[('x', 148), ('i', 109), ('j', 90), ('f', 89), ('p', 80), ('b', 79), ('w', 79), ('c', 67), ('s', 55), ('e', 49), ('v', 43), ('y', 40), ('q', 39), ('a', 38), ('m', 32), ('l', 28), ('o', 21), ('r', 20), ('z', 16), ('h', 11), ('g', 10), ('d', 6), ('k', 3), ('t', 2), ('u', 1)]


可以发现, x的出现次数最高,我们将x替换为e

In [35]:
set_dict('x', 'e')

#### 单词分析法
我们可以根据语言中的一些规律，分析单词来寻找映射关系
1. 单个字母一定为a 如果大写并出现在句首即为i
2. 三字母单词中出现频率最高的为the
3. 常出现的二字母单词为of, to, in, it, is, as, at, so, we, he, by, or, on, do, if, me, my, up, an, go, no, us, am等

In [36]:
#根据规则1,在密文中找到j单个出现,替换为a
set_dict('j', 'a')

#根据规则2
words_counter = {} #统计每个单词出现的次数
with open('ciphertext.txt', 'r') as f:
    text = f.read()
    text = convert_to_lower_case(text)
    #利用正则表达式提取出所有的单词
    import re
    words = re.findall(r'[a-z]+', text)
    #print(words)
    
    #统计单词出现次数
    for word in words:
        if word in words_counter:
            words_counter[word] += 1
        else:
            words_counter[word] = 1
    words_counter = sorted(words_counter.items(), key=lambda x: x[1], reverse=True)
    #print(words_counter)
    
    the_str = ""
    #找到长度为三的单词出现频率最高的，替换为the
    for word in words_counter:
        if len(word[0]) == 3:
            the_str = word[0]
            break
    #print(the_str)
    set_dict(the_str, 'the')

#print(dict)


现在我们已经获得了一些映射关系，所以写一个替换函数，将已知的映射关系替换为明文 其余用\*代替

In [37]:
def decrypt(string):
    string = convert_to_lower_case(string)
    result = ""
    for c in string:
        if c in dict: #已知
            result += dict[c]
        else:
            result += '*' #未知用*代替
    return result

尝试分析单词获得更多的映射关系

In [38]:
#分析对应长度的单词
def analysis(length):
    for word in words_counter:
        if len(word[0]) == length:
            print(decrypt(word[0]), word[1], word[0])
analysis(2)

t* 10 if
** 4 wc
** 3 fz
a* 2 jp
** 2 qf
** 2 wp
** 2 rm
*t 2 wi
*e 2 rx
** 1 fp
** 1 cf


In [39]:
#查常用二长度单词表得到部分映射关系
#of, to, in, it, is, as, at, so, we, he, by, or, on, do, if, me, my, up, an, go, no, us, am
#wi -> it
#if -> to

set_dict('wi', 'it')
set_dict('if', 'to')

analysis(2)

to 10 if
i* 4 wc
o* 3 fz
a* 2 jp
*o 2 qf
i* 2 wp
** 2 rm
it 2 wi
*e 2 rx
o* 1 fp
*o 1 cf


In [40]:
#联系 wc(i*) cf(*o) 可知c代表s 为is和so
set_dict('wc', 'is')
analysis(3)
print(dict)

the 12 iex
a** 4 jpq
a*e 3 jbx
**t 3 rli
a** 2 jpm
a*e 1 jox
*as 1 gjc
*a* 1 gjb
o*t 1 fli
its 1 wic
o*e 1 fpx
*a* 1 ajq
{'x': 'e', 'j': 'a', 'i': 't', 'e': 'h', 'w': 'i', 'f': 'o', 'c': 's'}


比较明显的二长度单词已经添加了映射关系，按理来说我们可以接着分析三长度单词。
在这里我们全部注释掉，不需要人工一个个找规律，试探能否找到映射关系。
因为在后文中提出了一种算法的改进，可以通过对长单词的模糊匹配来获得更多的映射关系。

In [41]:
#找规律 
# jbx(a*e) -> are
# jox(a*e) -> age
# fpx(o*e) -> one
# fli(o*t) -> out


# set_dict('jbx', 'are')
# set_dict('jox', 'age')
# set_dict('fpx', 'one')
# set_dict('fli', 'out')
# analysis(3)

In [42]:
#找规律 
# jpq(an*) -> and
# rli(*ut) -> but

# set_dict('jpq', 'and')
# set_dict('rli', 'but')
analysis(4)

*ith 3 gwie
*o*e 2 vfbx
s**h 2 clse
**o* 1 zbfv
o*** 1 fpym
sa*e 1 cjvx
*se* 1 lcxq
ha*e 1 ejhx
ha** 1 ejbq
that 1 ieji
e*e* 1 xhxp
ti*e 1 iwvx
tha* 1 iejp
*est 1 rxci


在代码的最开始版本 我人工识别出四长度甚至五六七长度的单词，这样的效率太低下，注释掉。

In [43]:
# set_dict('cjvx', 'same')
# set_dict('gwie', 'with')
# set_dict('xhxp', 'even')
# set_dict('ejhx', 'have')
# set_dict('clse', 'such')
analysis(5)

these 3 iexcx
**ea* 2 rbxjd
**io* 1 abwfb
state 1 cijix
si**e 1 cwpsx
*i*st 1 zwbci
*o*** 1 gfbyq
*a*** 1 sjbbm
*ase* 1 rjcxq
**o** 1 dpfgp
*ea*s 1 vxjpc
the*e 1 iexbx
e*ist 1 xkwci
*o*e* 1 afgxb


In [44]:
# set_dict('dpfgp', 'known')
# set_dict('gfbyq', 'world')
# set_dict('zwbci', 'first')
# set_dict('xkwci', 'exist')
analysis(6)
analysis(7)

se***e 3 cxslbx
*o*e** 2 vfqxbp
sha*e* 1 cejbxq
*ee*e* 1 pxxqxq
a**e*t 1 jqhxpi
*e*o*e 1 rxsfvx
theo** 1 iexfbm
a*o*** 1 jbflpq
*a*i** 1 vjdwpo
s*ste* 1 cmcixv
te**e* 1 ixbvxq
*aste* 1 zjcixb
*a**ot 1 sjppfi
**o*e* 1 rbfdxp
s*he*es 3 csexvxc
*essa*e 1 vxccjox
*e*o*e* 1 bxsfhxb
the*e** 1 iexbxrm
*e*so*s 1 axbcfpc
*etho*s 1 vxiefqc
*o***e* 1 sfvayxk
hea*i** 1 exjhwym
s*ie**e 1 cswxpsx
i*te*e* 1 wpixoxb
*e**i*e 1 bxtlwbx
a*a*te* 1 jqjaixq
e*a***e 1 xkjvayx


In [45]:
# set_dict('cmcixv', 'system')
# set_dict('axbcfpc', 'reasons')
# set_dict('sfvayxk', 'complex')
# set_dict('xkjvayx', 'example')
# set_dict('bxtlwbx', 'require')
# analysis(6)
# analysis(7)


In [46]:

print(len(dict)) 
print(dict) #打印字典
print(decrypt('wpzfbvjiwfp')) #information的密文

7
{'x': 'e', 'j': 'a', 'i': 't', 'e': 'h', 'w': 'i', 'f': 'o', 'c': 's'}
i**o**atio*


由上面可见，人力来模糊查找单词的效率低下，而且容易出错。
我们可以通过一些算法来寻找映射关系。

我们在一长度和二长度单词中，已经找到了部分映射关系：
`'x': 'e', 'j': 'a', 'i': 't', 'e': 'h', 'w': 'i', 'f': 'o', 'c': 's'`

猜想是否可以用这些进行模糊匹配，不再手工而是用代码实现后续的寻找呢？

例如information,根据已经找到的映射关系，不清楚的用\*代替，可以得到现在decrype的结果：
`i**o**atio*`
尽管人工识别有一定难度，但是利用正则进行模糊匹配（`\b`匹配单词边界）`\bi[a-z][a-z]o[a-z][a-z]atio[a-z]\b`：
在长度确定，再根据已知的部分是可以模糊识别出唯一的单词的。
我们就可以用这个单词更新映射表。

在这里，单词长度反而成为了一种标识，越长的单词，我们越能确定它的唯一性。与前面的单词分析法略微不同。

导入一个三千单词表，这可以确保大多数的单词都能在其中被找到。
我们对密文进行分词，然后对每个单词进行模糊匹配，如果能够找到唯一的对应单词，就更新映射表。
这样可以大大提高效率。
这部分的代码如下：

In [47]:
#正则表达式模糊查找单词
def find_in_list(string : str):
    string = r'\b' + string + r'\b'
    string = string.replace('*', '[a-z]')
    #print(string)
    with open('wordlist.txt', 'r') as f:
        text = f.read()
        result = re.findall(string, text)
        return result
# find_in_list("compl*x")

#长度大于2的单词，都拿来做匹配
lst = []
for word in words_counter:
    if len(word[0]) <= 2:
        continue
    lst.append(word[0])
#print(lst[:100])

#循环尝试匹配，直到字典中的单词全部匹配
while True:
    isFound = False
    for string in lst:
        origin = string
        string = decrypt(string)
        #print('de:', string)
        if '*' not in string: #如果没有*，说明已经匹配，跳过
            continue
        #print(string)
        result = find_in_list(string)
        if len(result) == 1:
            isFound = True
            set_dict(origin, result[0])
            print('set dict:')
            print('\tcipher word:', origin)
            print('\tmatched word:', result[0])
            print('\tdict length:', len(dict))
            print()
    if isFound == False:
        break

set dict:
	cipher word: gwie
	matched word: with
	dict length: 8

set dict:
	cipher word: wpzfbvjiwfp
	matched word: information
	dict length: 12

set dict:
	cipher word: cxslbx
	matched word: secure
	dict length: 14

set dict:
	cipher word: vfqxbp
	matched word: modern
	dict length: 15

set dict:
	cipher word: sfvalixb
	matched word: computer
	dict length: 16

set dict:
	cipher word: vxccjox
	matched word: message
	dict length: 17

set dict:
	cipher word: ixsepwtlx
	matched word: technique
	dict length: 18

set dict:
	cipher word: bxsfhxb
	matched word: recover
	dict length: 19

set dict:
	cipher word: gfbyq
	matched word: world
	dict length: 20

set dict:
	cipher word: sjbbm
	matched word: carry
	dict length: 21

set dict:
	cipher word: rxsfvx
	matched word: become
	dict length: 22

set dict:
	cipher word: sfvayxk
	matched word: complex
	dict length: 23

set dict:
	cipher word: rbfdxp
	matched word: broken
	dict length: 24



In [48]:
#最后进行打印输出
def decrypt_all(string):
    result = ""
    for c in string:
        if c in dict:
            result += dict[c]
        else:
            if c.lower() in dict: #注意大小写
                result += dict[c.lower()].upper()
            else:
                result += c
    return result

with open('origintext.txt', 'w') as fout:
    with open('ciphertext.txt', 'r') as fin:
        text = fin.read()
        fout.write(decrypt_all(text))

#打印映射表
print('明文与密钥的映射:')
sorted_dict = sorted(dict.items(), key=lambda x: x[1])
for item in sorted_dict:
    print('明文字母:', item[1], ' 密钥字母:', item[0])

明文与密钥的映射:
明文字母: a  密钥字母: j
明文字母: b  密钥字母: r
明文字母: c  密钥字母: s
明文字母: d  密钥字母: q
明文字母: e  密钥字母: x
明文字母: f  密钥字母: z
明文字母: g  密钥字母: o
明文字母: h  密钥字母: e
明文字母: i  密钥字母: w
明文字母: k  密钥字母: d
明文字母: l  密钥字母: y
明文字母: m  密钥字母: v
明文字母: n  密钥字母: p
明文字母: o  密钥字母: f
明文字母: p  密钥字母: a
明文字母: q  密钥字母: t
明文字母: r  密钥字母: b
明文字母: s  密钥字母: c
明文字母: t  密钥字母: i
明文字母: u  密钥字母: l
明文字母: v  密钥字母: h
明文字母: w  密钥字母: g
明文字母: x  密钥字母: k
明文字母: y  密钥字母: m


解密后的文本为：
```
Cryptography prior to the modern age was effectively synonymous with encryption, the conversion of information from a readable state to apparent nonsense. The originator of an encrypted message shared the decoding technique needed to recover the original information only with intended recipients, thereby precluding unwanted persons to do the same. Since the first World War and the advent of the computer, the methods used to carry out cryptology have become increasingly complex and its application more widespread.

Modern cryptography is heavily based on mathematical theory and computer science practice; cryptographic algorithms are designed around computational hardness assumptions, making such algorithms hard to break in practice by any adversary. It is theoretically possible to break such a system, but it is infeasible to do so by any known practical means. These schemes are therefore termed computationally secure; theoretical advances, e.g., improvements in integer factoriuation algorithms, and faster computing technology require these solutions to be continually adapted. There exist information-theoretically secure schemes that provably cannot be broken even with unlimited computing power--an example is the one-time pad--but these schemes are more difficult to implement than the best theoretically breakable but computationally secure mechanisms.
```

加密文本为：
```
Sbmaifobjaem abwfb if iex vfqxbp jox gjc xzzxsiwhxym cmpfpmvflc gwie xpsbmaiwfp, iex sfphxbcwfp fz wpzfbvjiwfp zbfv j bxjqjryx cijix if jaajbxpi pfpcxpcx. Iex fbwowpjifb fz jp xpsbmaixq vxccjox cejbxq iex qxsfqwpo ixsepwtlx pxxqxq if bxsfhxb iex fbwowpjy wpzfbvjiwfp fpym gwie wpixpqxq bxswawxpic, iexbxrm abxsylqwpo lpgjpixq axbcfpc if qf iex cjvx. Cwpsx iex zwbci Gfbyq Gjb jpq iex jqhxpi fz iex sfvalixb, iex vxiefqc lcxq if sjbbm fli sbmaifyfom ejhx rxsfvx wpsbxjcwpoym sfvayxk jpq wic jaaywsjiwfp vfbx gwqxcabxjq.

Vfqxbp sbmaifobjaem wc exjhwym rjcxq fp vjiexvjiwsjy iexfbm jpq sfvalixb cswxpsx abjsiwsx; sbmaifobjaews jyofbwievc jbx qxcwopxq jbflpq sfvalijiwfpjy ejbqpxcc jcclvaiwfpc, vjdwpo clse jyofbwievc ejbq if rbxjd wp abjsiwsx rm jpm jqhxbcjbm. Wi wc iexfbxiwsjyym afccwryx if rbxjd clse j cmcixv, rli wi wc wpzxjcwryx if qf cf rm jpm dpfgp abjsiwsjy vxjpc. Iexcx csexvxc jbx iexbxzfbx ixbvxq sfvalijiwfpjyym cxslbx; iexfbxiwsjy jqhjpsxc, x.o., wvabfhxvxpic wp wpixoxb zjsifbwujiwfp jyofbwievc, jpq zjcixb sfvaliwpo ixsepfyfom bxtlwbx iexcx cfyliwfpc if rx sfpiwpljyym jqjaixq. Iexbx xkwci wpzfbvjiwfp-iexfbxiwsjyym cxslbx csexvxc ieji abfhjrym sjppfi rx rbfdxp xhxp gwie lpywvwixq sfvaliwpo afgxb--jp xkjvayx wc iex fpx-iwvx ajq--rli iexcx csexvxc jbx vfbx qwzzwslyi if wvayxvxpi iejp iex rxci iexfbxiwsjyym rbxjdjryx rli sfvalijiwfpjyym cxslbx vxsejpwcvc.
```